In [0]:
# CARGA PRODUCTOS A BRONZE

In [0]:
import re
from pyspark.sql.functions import col

ruta_volumen_prod = "/Volumes/workspace/bronze/uploads/Productos.csv"

# 1. Leer el CSV
df_raw_prod = (spark.read.format("csv")
    .option("header", "true")
    .option("sep", ";")
    .option("encoding", "ISO-8859-1")
    .load(ruta_volumen_prod))

# 2. Limpieza de nombres de columnas
for old_name in df_raw_prod.columns:
    new_name = old_name.lower().replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')
    new_name = re.sub(r'[^a-z0-9]+', '_', new_name).strip('_')
    df_raw_prod = df_raw_prod.withColumnRenamed(old_name, new_name)

# 3. Eliminar columnas sin nombre (vienen como _c64, etc.)
cols_prod = [c for c in df_raw_prod.columns if not c.startswith("_c")]
df_final_bronze_prod = df_raw_prod.select(*cols_prod)

# 4. Guardar en Bronze
df_final_bronze_prod.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.bronze.productos")

print("✅ Tabla Productos en Bronze creada exitosamente.")